In [1]:
# Remove this to show output
%%capture 
!pip install git+https://github.com/mananbordia/rabbitx-python-client-test.git@main

from google.colab import drive
drive.mount('/content/drive')

## Working Solution

Steps:
1. Authenticate with wallet.
2. Send check opening balance
3. Set leverage to 1x
4. Create long and short order afte every 30 seconds
5. Size: 0.001; Market Id : BTC-USD
6. If current balance - opening balance < allowed negative delta, stop the system.
7. If any of the api calls fail we stop the system.
8. Maximum order count (default) : 50


In [43]:
from enum import Enum
from datetime import datetime
import json
import time
import random

from rabbitx import const
from rabbitx.client import Client, CandlePeriod, OrderSide, OrderType, OrderStatus, OpsType
from IPython.display import Audio, display


class MarketIds(Enum):
  ETH = "ETH-USD"
  BTC = "BTC-USD"
  SOL = "SOL-USD"
  ARB = "ARB-USD" # Only works on prod


def trigger_alarm():
  sound_file='./alarm.mp3'
  for _ in range(1):
    display(Audio(sound_file, autoplay=True))
    time.sleep(11)

def jprint(value):
  print(json.dumps(value,indent=2))

In [59]:
def create_client_and_onboard(private_key:str, api_url):
  client = Client(api_url=api_url, private_key=private_key)
  # Authenticate
  onboarding_resp = client.onboarding.onboarding()
  # jprint(onboarding_resp)
  return client, onboarding_resp

def set_leverage(client:Client, market_id:MarketIds, leverage:int):
  return client.account.set_leverage(market_id.value, leverage=leverage)

def place_order(client:Client,
                market_id:MarketIds,
                size:int, 
                side:OrderSide,
                txn_id:int=0,
                price:float=0.0001,
                type_:OrderType=OrderType.MARKET):
  
  # Todo: Check for unusual cases

  # Fixme: Check if price not provided and is a limit order
  if price == 0.0001 and type_ == OrderType.LIMIT:
    raise Exception("You need to provide price while placing LIMIT order")

  market_price = get_current_market_price(client,market_id)
  total_amount = market_price * size

  if total_amount > 500:
    raise Exception("Unusual order with total amount > $500")

  jprint("Txn-{} {}: {} initiated | amount {} | size {} | market price ${}".format(txn_id,market_id.value,side.value,total_amount,size,market_price))
  order_resp = client.orders.create(
      market_id=market_id.value,
      size=size,
      side=side,
      type_=type_,
      price=price
  )
  jprint("Txn-{} {}: {} placed".format(txn_id,market_id.value,side.value))
  return order_resp

def get_balance_history(client:Client):
  return client.balance.list()

def get_balance(client:Client):
  return float(client.account.get().get('balance'))

def print_recent_balance_change(ops_type:OpsType, history):
  ops_type = ops_type.value
  ops_history = list(filter(lambda x: x.get('ops_type')==ops_type, history))
  mapped_values = list(map(lambda x: {
      "amount": x.get('amount'),
      "at": datetime.utcfromtimestamp(19800 + int(x.get('timestamp')/1000000)).strftime('%d %b, %Y - %H:%M:%S')},
      ops_history))
  recent_entry = mapped_values[:1]
  if len(recent_entry) == 1:
    jprint("{} history -> amount: {} | at = {}".format(ops_type, recent_entry[0].get('amount'), recent_entry[0].get('at')))
  else:
    jprint("No {} history found".format(ops_type))

def show_balance_details(client:Client, opening_balance:float):
  # Info: We can check fees, funding, withdraw, deposit, pnl
  current_balance = get_balance(client)
  jprint("Current Balance: {}".format(current_balance))
  jprint("Delta in balance (CurB - OpenB): {}".format(current_balance - opening_balance))
  balance_history = get_balance_history(client)
  print_recent_balance_change(OpsType.FEE, balance_history)
  print_recent_balance_change(OpsType.PNL, balance_history)

  if current_balance - opening_balance <= -10:
    raise Exception("You have lost over $10. Stopping system")
  # print_recent_balance_change(OpsType.FUNDING, balance_history)

def get_position_list(client:Client):
  return client.positions.list()

def print_position_data(position):
  market_id = position.get('market_id')
  side = position.get('side')
  size = float(position.get('size'))
  unrealized_pnl = float(position.get('unrealized_pnl'))
  jprint("{}-> {} | size: {} | unrealised pnl: {}".format(market_id,side,size,round(unrealized_pnl,2)))

def show_position_details(client:Client):
  pos_list = get_position_list(client)
  mapped_pos_list = list(map(lambda x: {
      'market_id': x.get('market_id'),
      'side': x.get('side'),
      'size': x.get('size'),
      'unrealized_pnl': x.get('unrealized_pnl')
  }, pos_list))
  for pos in mapped_pos_list:
    print_position_data(pos)

def get_current_market_price(client:Client, market_id:MarketIds):
  market_list = client.markets.list()
  current_price = list(filter(lambda x: x.get('id') == market_id.value, market_list))[0].get('market_price')
  return float(current_price)

def close_position(client:Client, market_id:MarketIds, txn_id:int=0):
  pos_list = get_position_list(client)
  pos = next((filter(lambda x: x.get('market_id')==market_id.value,pos_list)), None)
  if pos:
    size=float(pos.get('size'))
    side=next((osize for osize in list(OrderSide) if pos.get('side') != osize.value), None)
    place_order(client, market_id, size, side, txn_id=txn_id)
  else:
    jprint("Position already closed.")
    
def get_total_trading_volm(client:Client):
    return float(client.profile.get().get('cum_trading_volume'))


In [60]:
def execute_v1(private_key, market_ids:list[MarketIds], weights:list[int], amount_range:list[float], leverage:int=1, max_order:int=10, api_url=const.DEV_URL):
  try:
    client, _ = create_client_and_onboard(private_key,api_url)
    
    jprint("Running on {}".format(api_url))

    opening_balance = get_balance(client)
    jprint("Opening Balance: {}".format(opening_balance))
    opening_total_trading_volm = get_total_trading_volm(client)
    jprint("Cummulative Trading Volm: {}".format(opening_total_trading_volm))

    last_total_trading_volm = opening_total_trading_volm
    
    no_change_trading_volm_count = 0
    order_count = 0
    if len(amount_range) != 2 or amount_range[0] > amount_range[1]:
      raise Exception("Provide proper amount range: {}-{}".format(amount_range[0], amount_range[1]))

    while(True):
      print('\n\n')
      jprint("################## Start ##################")
      
      # Try to set leverage to req_leverage
      while(True):
        [market_id] = random.choices(market_ids, weights=weights)
        cur_leverage = int(set_leverage(client, market_id, leverage).get('leverage').get(market_id.value))
        if cur_leverage != leverage:
          jprint("Unable to set leverage. Trying again. Current leverage: {}x".format(cur_leverage))
        else:
          jprint("Leverage set to {}x".format(cur_leverage))
          break

      order_count += 1 # Optimistic
      # side = OrderSide.SHORT if order_count % 2 else OrderSide.LONG
        
      curr_balance = get_balance(client)
      amount_range[1] = min(curr_balance - 0.5, amount_range[1])
      amount_range[0] = min(curr_balance - 0.5, amount_range[0])

      if amount_range[0] < 0:
        raise Exception("Insufficient balance")
      
      order_amount = random.uniform(amount_range[0], amount_range[1]) * cur_leverage
      cur_market_price = get_current_market_price(client, market_id)
      size = round(order_amount / cur_market_price,3)
      side = OrderSide.LONG
      # Info: Placing only market order
      place_order(client,market_id,size,side,txn_id=order_count)
    
      jprint("-------------Closing open position-----------")
      show_position_details(client)
      close_position(client, market_id, order_count)
      show_position_details(client)
      jprint("-----------------Stats---------------------")
      show_balance_details(client, opening_balance)
      curr_total_trading_volm = get_total_trading_volm(client)

      delta_trading_volm = curr_total_trading_volm - opening_total_trading_volm
      curr_delta_trading_volm = curr_total_trading_volm - last_total_trading_volm
      last_total_trading_volm = curr_total_trading_volm 
      
      if curr_delta_trading_volm < 0.1:
        no_change_trading_volm_count += 1
        if no_change_trading_volm_count == 3:
          raise Exception("No change in trading volm. Please check...")
      else:
        no_change_trading_volm_count = 0

      jprint("Total Volm Gain (approx) : {}".format(delta_trading_volm))
      if order_count >= max_order:
        jprint("Max order limit reached. Order Count: {}.".format(order_count))
        break

      sleep_time = random.randint(10, 20)
      jprint("Sleeping for {} seconds before placing next order".format(sleep_time))
      time.sleep(sleep_time)

  except KeyboardInterrupt as ke:
    print("Stopping as per user's request")
  except Exception as e:
    print("Something went wrong: ",e)
    trigger_alarm()

# Execute this to place orders

In [64]:
# Print IP address
print("IP address:")
!curl ipecho.net/plain
print('\n')
# Change this with your private key
private_key = '0x0000000000000000000000000000000000000000000000000000000000000001'
# Please be careful about size
# Todo: write algo which works via USDTs
amount_range = [8.0, 10.0]
execute_v1(private_key,
           [MarketIds.ETH, MarketIds.BTC],
           [1,1],
           amount_range,
           leverage=20,
           max_order=100,
           api_url=const.URL)

IP address:
49.207.195.245
"Running on https://api.prod.rabbitx.io"
"Opening Balance: 2.98136498"
"Cummulative Trading Volm: 10180.7586"



"################## Start ##################"
"Leverage set to 20x"
"Txn-1 ETH-USD: long initiated | amount 50.13765 | size 0.027 | market price $1856.95"
"Txn-1 ETH-USD: long placed"
"-------------Closing open position-----------"
"BTC-USD-> long | size: 0.002 | unrealised pnl: -0.0"
"Position already closed."
"BTC-USD-> long | size: 0.002 | unrealised pnl: -0.0"
"-----------------Stats---------------------"
"Current Balance: 2.98136498"
"Delta in balance (CurB - OpenB): 0.0"
"fee history -> amount: -0.039039 | at = 09 Apr, 2023 - 00:31:25"
"pnl history -> amount: -0.014 | at = 09 Apr, 2023 - 00:31:04"
"Total Volm Gain (approx) : 0.0"
"Sleeping for 19 seconds before placing next order"
Stopping as per user's request


In [63]:
!curl ipecho.net/plain

49.207.195.245

## Experiment-1 with 2 orders (immediate close)
- Both were placed as long order and immediately closed.
- Time dif b/w both the long orders  20 - 40 seconds 

In [ ]:
opening_bal = 7.29844
end_bal = 7.21411587
total_cost = opening_bal - end_bal
vol_start = 11355.93
vol_end = 11585.29
vol_gain = vol_end - vol_start
total_time = 30

cost_per_vol_unit = total_cost / vol_gain
print("Total Cost of Experiment: {} USD".format(total_cost))
print("Total Volume Gain in Experiment: {} USD".format(vol_gain))
print("Cost req for 100k vol: {} USD".format(cost_per_vol_unit * 100000))
print("Total Time Required ~{} seconds".total_time)

Total Cost of Experiment: 0.08432413000000061 USD
Total Volume Gain in Experiment: 229.36000000000058 USD
Cost req for 100k vol: 36.764967736309906 USD


## Experiment-2 with 2 orders (close with wait)
- Both long orders.
- Position for was closed after 10 - 20 of wait time.
- Time dif b/w both the long orders  20 - 40 seconds 

In [ ]:
opening_bal = 7.13179987
end_bal = 6.965219
total_cost = opening_bal - end_bal
vol_start = 11585.29
vol_end = 11814.69
vol_gain = vol_end - vol_start
total_time = 30

cost_per_vol_unit = total_cost / vol_gain
print("Cost req for 100k vol: {} USD".format(cost_per_vol_unit * 100000))

Cost req for 100k vol: 72.61589799476899 USD


In [ ]:
cost_per_vol_unit

0.00036764967736309905

In [10]:
random.randint(0,1)

1

In [33]:
random.choices([1,2,3], weights=[10,10,1])

[2]